In [1]:
!pip install kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle datasets download -d snehaanbhawal/resume-dataset
!unzip resume-dataset.zip

 62% 39.0M/62.5M [00:00<00:00, 209MB/s]
100% 62.5M/62.5M [00:00<00:00, 217MB/s]


In [6]:
import re
import pandas as pd
xdf=pd.read_csv('/content/Resume/Resume.csv')
jobs = pd.DataFrame()
jobs['job'] = xdf.Resume_str
jobs['Category']=xdf.Category
jobs.columns

Index(['job', 'Category'], dtype='object')

In [7]:

def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText)
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText


from sklearn.preprocessing import LabelEncoder
data=pd.DataFrame()
data[['Resume','Category']]=jobs[['job','Category']]
le = LabelEncoder()
data['Category'] = le.fit_transform(data['Category'])
data['Resume'] = data.Resume.apply(lambda x: cleanResume(x))
data['Resume'] = data.Resume.map(str. lower)
data

,Resume,Category
0,hr administrator marketing associate hr admin...,19
1,hr specialist us hr operations summary versat...,19
2,hr director summary over 20 years experience ...,19
3,hr specialist summary dedicated driven and dy...,19
4,hr manager skill highlights hr skills hr depa...,19
...,...,...
2479,rank sgt e 5 non commissioned officer in char...,6
2480,government relations communications and organ...,6
2481,geek squad agent professional profile it supp...,6
2482,program director office manager summary highl...,6


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(data, *['Category'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(data, *['Category'], **{})
chart

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

requiredText = data['Resume'].values
requiredTarget = data['Category'].values

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    stop_words='english',
    max_df=0.8,                # Ignore terms that appear in more than 80% of documents
    min_df=2 #min 2 occurence
)
word_vectorizer.fit(requiredText)
WordFeatures = word_vectorizer.transform(requiredText)
print ("Feature completed .....")

X_train,X_test,y_train,y_test = train_test_split(WordFeatures,requiredTarget,random_state=42, test_size=0.2,
                                                 shuffle=True, stratify=requiredTarget)

Feature completed .....


76% accuracy

In [21]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

# Creating the Gradient Boosting Classifier object
gbc = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Fitting the model on the training data
gbc.fit(X_train, y_train)

# Predicting the target variable on the test data
y_pred = gbc.predict(X_test)

# Printing the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Printing the confusion matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)


Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        24
           1       0.78      0.88      0.82        24
           2       0.78      0.54      0.64        13
           3       0.47      0.42      0.44        19
           4       0.63      0.57      0.60        21
           5       0.50      0.29      0.36         7
           6       0.95      0.75      0.84        24
           7       0.81      0.57      0.67        23
           8       0.67      0.50      0.57         4
           9       0.64      0.67      0.65        24
          10       0.90      0.79      0.84        24
          11       0.95      0.86      0.90        22
          12       0.67      0.78      0.72        23
          13       0.88      1.00      0.93        21
          14       0.71      0.53      0.61        19
          15       0.80      0.83      0.82        24
          16       0.83      0.83      0.83        24
    